In [1]:
import datetime
from ib_insync import *
import pandas as pd
from arcticdb import Arctic, QueryBuilder, LibraryOptions

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

### Connect to IB

In [2]:
ib = connect_to_IB(clientid=40)
ac = initialize_db('db')

In [3]:
ac.list_libraries()

['general', 'pnl', 'portfolio', 'spx500', 'stocks']

In [4]:
# ac.delete_library('portfolio')

In [5]:
# pm = PortfolioManager(ib)
# Library(Arctic(config=LMDB(path=c:\Users\Johannes.Lange\pythonscripts\ATS\data_and_research\data_and_research\db)), path=portfolio, storage=lmdb_storage)
pm = PortfolioManager(ib,arctic=ac)


In [6]:
df = pm.portfolio_library.read(pm.account_id).data
df

,symbol,strategy,asset class,position,timestamp,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,AAPL,,STK,37.0,2024-06-03 14:53,3.280909,180.260541,192.889999,7.006225,"Stock(conId=265598, symbol='AAPL', right='0', ...",...,,False,,7136.930000,467.290000,USD,0.00,DU3180649,6581.940755,1.084320
1,CNR,,STK,-699.0,2024-06-03 14:53,-40.957544,174.287153,173.643005,0.369590,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-121376.460000,450.260000,CAD,0.00,DU3180649,-82166.293778,1.477205
2,SIE,,STK,1.0,2024-06-03 14:53,0.089052,181.208000,178.647812,-1.412845,"Stock(conId=14217, symbol='SIE', right='0', pr...",...,,False,,178.650000,-2.560000,EUR,-8.32,DU3180649,178.650000,1.000000
3,VOW,,STK,-176.0,2024-06-03 14:53,-11.584126,141.129400,132.041504,6.439407,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-23239.300000,1599.470000,EUR,0.00,DU3180649,-23239.300000,1.000000
4,AAPL,,STK,37.0,2024-06-03 14:58,3.280816,180.260541,192.899994,7.011769,"Stock(conId=265598, symbol='AAPL', right='0', ...",...,,False,,7137.299774,12.639453,USD,0.00,DU3180649,6581.940755,1.084320
5,CNR,,STK,-699.0,2024-06-03 14:58,-40.956383,174.287153,173.643005,0.369590,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-121376.460740,-0.644148,CAD,0.00,DU3180649,-82166.293778,1.477205
6,VOW,,STK,-176.0,2024-06-03 14:58,-11.583797,141.129400,132.041504,6.439407,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-23239.304686,-9.087896,EUR,0.00,DU3180649,-23239.300000,1.000000
7,AAPL,,STK,37.0,2024-06-03 14:58,3.280816,180.260541,192.899994,7.011769,"Stock(conId=265598, symbol='AAPL', right='0', ...",...,,False,,7137.299774,12.639453,USD,0.00,DU3180649,6581.940755,1.084320
8,CNR,,STK,-699.0,2024-06-03 14:58,-40.956383,174.287153,173.643005,0.369590,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-121376.460740,-0.644148,CAD,0.00,DU3180649,-82166.293778,1.477205
9,SIE,,STK,1.0,2024-06-03 14:53,0.089052,181.208000,178.647812,-1.412845,"Stock(conId=14217, symbol='SIE', right='0', pr...",...,,False,,178.650000,-2.560000,EUR,-8.32,DU3180649,178.650000,1.000000


In [7]:
df.iloc[11][5:]

% of nav                                                          0.0
averageCost                                                    178.64
marketPrice                                                    178.64
pnl %                                                             0.0
contract            Stock(conId=14217, symbol='SIE', exchange='SMA...
trade               Trade(contract=Stock(conId=14217, symbol='SIE'...
trade_context       Trade(contract=Stock(conId=14217, symbol='SIE'...
open_dt                                                    2024-06-03
close_dt                                                             
deleted                                                         False
delete_dt                                                            
marketValue                                                       0.0
unrealizedPNL                                                     0.0
currency                                                          EUR
realizedPNL         

In [63]:
df.contract.to_list()

["Stock(conId=265598, symbol='AAPL', right='0', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS')",
 "Stock(conId=4458894, symbol='CNR', right='0', primaryExchange='TSE', currency='CAD', localSymbol='CNR', tradingClass='CNR')",
 "Stock(conId=14232, symbol='VOW', right='0', primaryExchange='IBIS', currency='EUR', localSymbol='VOW', tradingClass='XETRA')",
 "Stock(conId=265598, symbol='AAPL', right='0', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS')",
 "Stock(conId=4458894, symbol='CNR', right='0', primaryExchange='TSE', currency='CAD', localSymbol='CNR', tradingClass='CNR')",
 "Stock(conId=14232, symbol='VOW', right='0', primaryExchange='IBIS', currency='EUR', localSymbol='VOW', tradingClass='XETRA')",
 "Stock(conId=265598, symbol='AAPL', right='0', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS')",
 "Stock(conId=4458894, symbol='CNR', right='0', primaryExchange='TSE', currency='CAD', loc

In [62]:
# A TEST to see if we can do eval(df.trade) and find out if trade is in column
last_trade in df.contract.to_list()


True

In [19]:
latest_positions = (df['symbol'] == 'VOW') & \
                        (df['asset class'] == 'STK') & \
                        (df['strategy'] == '') & \
                        (df['deleted'] == False)

existing_position = df[latest_positions].sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class','position']).last().reset_index()

In [28]:
df_ac_active = df[df['deleted'] != True].copy()
df_ac_active = df_ac_active.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
existing_position = df_ac_active[(df_ac_active['symbol'] == 'VOW') & (df_ac_active['asset class'] == 'STK') 
                                & (df_ac_active['strategy'] == '')]
existing_position

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
3,VOW,,STK,2024-06-01 20:39,765.0,50.168699,140.97045,131.600006,-6.647098,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,100674.004666,-7168.389584,EUR,0.0,DU3180649,100674.004666,1.0


In [41]:
if type(existing_position) == pd.DataFrame:
            filter_condition = (df['symbol'] == existing_position.iloc[0]['symbol']) & \
                        (df['asset class'] == existing_position.iloc[0]['asset class']) & \
                        (df['position'] == existing_position.iloc[0]['position']) & \
                        (df['strategy'] == existing_position.iloc[0]['strategy']) & \
                        (df['deleted'] == False)
            
df = df[filter_condition][-1]
df

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
2,VOW,,STK,2024-05-27 18:12,765.0,53.758911,140.97045,141.081924,0.079076,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,107927.670000,85.280000,EUR,-3230.09,DU3180649,107927.670000,1.0
5,VOW,,STK,2024-05-27 18:13,765.0,53.758911,140.97045,141.081924,0.079076,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,107927.672204,0.111474,EUR,-3230.09,DU3180649,107927.670000,1.0
9,VOW,,STK,2024-05-27 22:04,765.0,53.758364,140.97045,141.600006,0.446587,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,108324.004666,0.629556,EUR,-3230.09,DU3180649,107927.672204,1.0
13,VOW,,STK,2024-05-27 22:10,765.0,53.955770,140.97045,141.600006,0.446587,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,108324.004666,0.629556,EUR,-3230.09,DU3180649,108324.004666,1.0
17,VOW,,STK,2024-05-28 10:23,765.0,54.095603,140.97045,141.199997,0.162833,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,108017.997667,0.229547,EUR,-3230.09,DU3180649,108324.004666,1.0
23,VOW,,STK,2024-05-28 10:55,765.0,53.977328,140.97045,142.039429,0.758300,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,108660.162955,1.068979,EUR,-3230.09,DU3180649,108017.997667,1.0
29,VOW,,STK,2024-05-28 11:25,765.0,54.361758,140.97045,142.739990,1.255256,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,109196.092541,1.769540,EUR,-3230.09,DU3180649,108660.162955,1.0
36,VOW,,STK,2024-05-29 11:27,765.0,53.865656,140.97045,139.963074,-0.714601,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,107071.751419,-1.007376,EUR,-3230.09,DU3180649,109196.092541,1.0
43,VOW,,STK,2024-05-29 12:22,765.0,52.601565,140.97045,139.459991,-1.071472,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,106686.893459,-1155.500791,EUR,0.00,DU3180649,106686.893459,1.0
48,VOW,,STK,2024-05-29 17:15,765.0,52.533687,140.97045,139.430496,-1.092395,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,106664.329593,-1.539954,EUR,0.00,DU3180649,106686.893459,1.0


In [48]:
df.tail(1)

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
127,VOW,,STK,2024-06-01 20:39,765.0,50.168699,140.97045,131.600006,-6.647098,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,100674.004666,-7168.389584,EUR,0.0,DU3180649,100674.004666,1.0


Peer closed connection.


In [35]:
df.index

MultiIndex([('VOW', '', 'STK', 765.0)],
           names=['symbol', 'strategy', 'asset class', 'position'])

In [6]:
df = pm.get_ib_positions_for_gui()
df

0      37.0
1   -1586.0
2       1.0
3     765.0
Name: position, dtype: float64
          timestamp symbol asset class  position   % of nav  averageCost  \
0  2024-05-30 22:03   AAPL         STK      37.0   3.201621   180.260541   
1  2024-05-30 22:03    CNR         STK    -699.0 -39.232571   174.287153   
2  2024-05-30 22:03    SIE         STK       1.0   0.086285   183.500000   

   marketPrice     pnl % strategy  \
0   191.289993  6.118617            
1   169.744064  2.606669            
2   176.119995 -4.021801            

                                            contract  ... close_dt deleted  \
0  Stock(conId=265598, symbol='AAPL', right='0', ...  ...            False   
1  Stock(conId=4458894, symbol='CNR', right='0', ...  ...            False   
2  Stock(conId=14217, symbol='SIE', right='0', pr...  ...            False   

  delete_dt  marketValue unrealizedPNL  currency  realizedPNL    account  \
0                7077.73        408.09       USD          0.0  DU3180649   
1 

,symbol,asset class,position,% of nav,currency,marketPrice,averageCost,pnl %,strategy
2,CNR,STK,887.0,49.784392,CAD,169.744064,174.740231,-2.859197,
1,CNR,STK,-1586.0,-88.831453,CAD,169.744064,174.540546,2.748061,
4,VOW,STK,765.0,49.330736,EUR,131.622360,140.970450,-6.631241,
5,VOW,STK,-941.0,-60.680029,EUR,131.622360,141.000179,6.650927,
0,AAPL,STK,37.0,3.206709,USD,191.289993,180.260541,6.118617,
3,SIE,STK,1.0,0.086295,EUR,176.119995,183.500000,-4.021801,


In [7]:
position_to_merge = ('VOW','',765)
target_position = ('VOW','',-941)

In [27]:
df_ac = pm.portfolio_library.read(pm.account_id).data

df_ac_active = df_ac[df_ac['deleted'] != True].copy()
latest_active_entries = df_ac_active.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class','position']).last().reset_index()
latest_active_entries
latest_active_entries.iloc[4:, 7:]

,marketPrice,pnl %,contract,trade,open_dt,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
4,131.62236,6.650927,"Stock(conId=14232, symbol='VOW', right='0', pr...",,2024-05-30,,False,,-123856.640995,8824.527655,EUR,0.0,DU3180649,-123856.640995,1.0
5,131.62236,-6.631241,"Stock(conId=14232, symbol='VOW', right='0', pr...",,2024-05-30,,False,,100691.105591,-9.348090,EUR,0.0,DU3180649,100691.105591,1.0


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 10197, reqId 5: Keine Marktdaten w\u00e4hrend konkurrierender Live-Sitzung, contract: Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')
Error 10197, reqId 7: Keine Marktdaten w\u00e4hrend konkurrierender Live-Sitzung, contract: Forex('EURCAD', conId=15016068, exchange='IDEALPRO', localSymbol='EUR.CAD', tradingClass='EUR.CAD')
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu folgenden Datenzentren hergestellt: . Keine Verbindung zu folgenden Datenzentren: usfarm.nj; eufarm; cafarm; cashfarm; usfarm; euhmds; cashhmds; ushmds; fundfarm; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde

In [9]:
row_to_merge = latest_active_entries[(latest_active_entries['symbol'] == position_to_merge[0]) 
                & (latest_active_entries['strategy'] == position_to_merge[1]) 
                & (latest_active_entries['position'] == position_to_merge[2])].reset_index(drop=True)
target_row = latest_active_entries[(latest_active_entries['symbol'] == target_position[0]) 
                & (latest_active_entries['strategy'] == target_position[1]) 
                & (latest_active_entries['position'] == target_position[2])].reset_index(drop=True)
target_row

,symbol,strategy,asset class,position,timestamp,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,VOW,,STK,-941.0,2024-05-30 22:03,-60.680029,141.000179,131.62236,6.650927,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-123856.640995,8824.527655,EUR,0.0,DU3180649,-123856.640995,1.0


In [30]:
closed = True if agg_df.loc[0,'position'] == 0 else False
closed

False

In [36]:

agg_df = target_row.copy()
agg_df.loc[0,'position'] += row_to_merge.loc[0,'position']

# Check if position is 0 then trade was closed, we can calculate PnL and delete both positions
closed = True if agg_df.loc[0,'position'] == 0 else False

agg_df.loc[0,'timestamp'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
agg_df.loc[0,'marketValue_base'] += row_to_merge.loc[0,'marketValue_base']
agg_df.loc[0,'% of nav'] = ( agg_df.loc[0,'marketValue_base'] / pm.total_equity ) * 100
agg_df.loc[0,'averageCost'] = ( (row_to_merge.loc[0,'averageCost'] * row_to_merge.loc[0,'position']) + 
                                (target_row.loc[0,'averageCost'] * target_row.loc[0,'position']) ) / agg_df.loc[0,'position']
agg_df.loc[0,'pnl %'] = ( (agg_df.loc[0,'marketPrice'] - agg_df.loc[0,'averageCost']) * agg_df.loc[0,'position'] )/ agg_df.loc[0,'marketValue_base']
# skip contract

# store trade context
if not agg_df.loc[0,'trade']:
    trade_context = row_to_merge.loc[0,'trade']
elif not type(eval(agg_df.loc[0,'trade'])) == list:
    trade_context = [agg_df.loc[0,'trade'], row_to_merge.loc[0,'trade'] ]
else:
    trade_context = str(eval(agg_df.loc[0,'trade']).append(row_to_merge.loc[0,'trade']))
agg_df.loc[0,'trade'] = trade_context

# take first day of opening trade
agg_df.loc[0,'open_dt'] = agg_df.loc[0,'open_dt'] if datetime.datetime.fromisoformat(agg_df.loc[0,'open_dt']) <= datetime.datetime.fromisoformat(row_to_merge.loc[0,'open_dt']) else row_to_merge.loc[0,'open_dt']
agg_df.loc[0,'close_dt'] = datetime.date.today().isoformat() if closed else ''
agg_df.loc[0, 'marketValue'] = agg_df.loc[0, 'marketPrice'] * agg_df.loc[0, 'position']
agg_df.loc[0, 'unrealizedPNL'] = (agg_df.loc[0, 'marketPrice'] - agg_df.loc[0, 'averageCost']) * agg_df.loc[0, 'position']

agg_df.loc[0, 'realizedPNL'] = (agg_df.loc[0, 'marketPrice'] - agg_df.loc[0, 'averageCost']) * agg_df.loc[0, 'position'] if closed else 0.0


In [35]:
datetime.datetime.fromisoformat(agg_df.loc[0,'open_dt']) <= datetime.datetime.fromisoformat(row_to_merge.loc[0,'open_dt'])

True

In [22]:
trade_context

''

In [19]:
eval(agg_df.loc[0,'contract'])

Stock(conId=14232, symbol='VOW', right='0', primaryExchange='IBIS', currency='EUR', localSymbol='VOW', tradingClass='XETRA')

In [23]:
agg_df.loc[:,'pnl %':]

,pnl %,contract,trade,open_dt,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,-0.07223,"Stock(conId=14232, symbol='VOW', right='0', pr...",,2024-05-30,,False,,-123856.640995,8824.527655,EUR,0.0,DU3180649,-23165.535404,1.0


In [63]:
agg_df.contract[0]

"Stock(conId=14232, symbol='VOW', right='0', primaryExchange='IBIS', currency='EUR', localSymbol='VOW', tradingClass='XETRA')"

In [ ]:
trade_dict = {'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M'),
            'symbol': trade.contract.symbol,
            'asset class': trade.contract.secType,
            'position': trade.order.totalQuantity,
            '% of nav': 0.0, # to be calculated
            'averageCost': trade.orderStatus.avgFillPrice,
            'marketPrice': trade.orderStatus.lastFillPrice,
            'pnl %': 0.0, # to be calculated
            'strategy': strategy_symbol,
            'contract': str(trade.contract),
            'trade': str(trade),
            'open_dt':datetime.date.today().isoformat(),
            'close_dt': '',
            'deleted': False,
            'delete_dt': '',
            'marketValue': 0.0, # to be calculated
            'unrealizedPNL': 0.0, # to be calculated
            'currency':trade.contract.currency,
            'realizedPNL': 0.0, # to be calculated
            'account': trade.fills[0].execution.acctNumber,
            'marketValue_base': 0.0, # to be calculated
            'fx_rate': 0.0}
        except Exception as e:
            print(f"Error processing trade: {e}")


In [24]:

# print("print from match_ib_positions_with_arcticdb")
if pm.account_id in pm.portfolio_library.list_symbols():
    df_ac = pm.portfolio_library.read(f'{pm.account_id}').data
    # Filter out deleted entries before comparing
    df_ac_active = df_ac[df_ac['deleted'] != True].copy()
    latest_positions_in_ac = df_ac_active.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class','position']).last().reset_index()
    print(latest_positions_in_ac['position'])
else:
    print(f"Msg from function 'match_ib_positions_with_arcticdb': No Symbol {pm.account_id} in library 'portfolio'")
    df_ib = pm.get_positions_from_ib()

df_ib = pm.get_positions_from_ib()
df_merged = pd.DataFrame()

# Iterate through the positions obtained from Interactive Brokers
for index, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']

    # Filter the ArcticDB DataFrame for entries with the same symbol and asset class
    strategy_entries_in_ac = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & (latest_positions_in_ac['asset class'] == asset_class)]
    sum_of_position_entries = strategy_entries_in_ac['position'].sum()
    print(strategy_entries_in_ac)
    if strategy_entries_in_ac.empty: # no database entry, add position
        df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)

    elif len(strategy_entries_in_ac) == 1:
        if total_position == sum_of_position_entries:
            # the positions match, we should only update fields relevant for marketdata in ac
            strategy_entries_in_ac = pm.update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
        else:
            #update marketdata relevant fields for the existing db entry
            strategy_entries_in_ac = pm.update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
            
            # handle the residual and concat to df_merged
            residual = pm.handle_residual(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, residual], ignore_index=True)

    elif len(strategy_entries_in_ac) > 1:
        if total_position == sum_of_position_entries:
            print("len(strategy_entries_in_ac) > 1 and total_position == sum_of_position_entries")
            strategy_entries_in_ac = pm.update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
        else:
            print("len(strategy_entries_in_ac) > 1 and total_position != sum_of_position_entries")
            #update marketdata relevant fields for the existing db entry
            strategy_entries_in_ac = pm.update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
            
            # handle the residual and concat to df_merged
            residual = pm.handle_residual(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, residual], ignore_index=True)
    

0      37.0
1   -1586.0
2     887.0
3       1.0
4    -941.0
5     765.0
Name: position, dtype: float64
  symbol strategy asset class  position         timestamp  % of nav  \
0   AAPL                  STK      37.0  2024-05-30 15:27   3.20037   

   averageCost  marketPrice    pnl %  \
0   180.260541   190.460007  5.65818   

                                            contract  ... close_dt deleted  \
0  Stock(conId=265598, symbol='AAPL', right='0', ...  ...            False   

  delete_dt  marketValue unrealizedPNL  currency  realizedPNL    account  \
0            7047.020248     10.199466       USD          0.0  DU3180649   

   marketValue_base   fx_rate  
0       6539.153418  1.086015  

[1 rows x 22 columns]
  symbol strategy asset class  position         timestamp   % of nav  \
1    CNR                  STK   -1586.0  2024-05-30 15:27 -88.319326   
2    CNR                  STK     887.0  2024-05-30 15:27  49.468440   

   averageCost  marketPrice     pnl %  \
1   174.540546   1

ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
df_merged

In [8]:
df = pm.portfolio_library.read('test').data


In [13]:
for col in df.columns:
    print(f"{col}: {type(df[col][0])}")

timestamp: <class 'str'>
symbol: <class 'str'>
asset class: <class 'str'>
position: <class 'numpy.float64'>
% of nav: <class 'numpy.float64'>
averageCost: <class 'numpy.float64'>
marketPrice: <class 'numpy.float64'>
pnl %: <class 'numpy.float64'>
strategy: <class 'str'>
contract: <class 'str'>
trade: <class 'str'>
open_dt: <class 'str'>
close_dt: <class 'str'>
deleted: <class 'numpy.bool_'>
delete_dt: <class 'str'>
marketValue: <class 'numpy.float64'>
unrealizedPNL: <class 'numpy.float64'>
currency: <class 'str'>
realizedPNL: <class 'numpy.float64'>
account: <class 'str'>
marketValue_base: <class 'numpy.float64'>
fx_rate: <class 'numpy.float64'>


In [5]:
df = pm.get_positions_from_ib()
if df.empty:
    print(" empty")

In [7]:
ac.get_library('portfolio').read('test').data.tail()

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
33,CNR,,STK,2024-05-29 11:27,887.0,51.490026,174.740231,169.473999,-3.013749,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,150323.437113,-5.266232,CAD,0.00,DU3180649,104380.230712,1.48183
34,CNR,,STK,2024-05-29 11:27,-1586.0,-89.489351,174.540546,169.473999,2.902791,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-268785.762414,8035.542828,CAD,0.00,DU3180649,-181412.203056,1.48163
35,SIE,,STK,2024-05-29 11:27,1.0,0.089241,183.500000,176.932007,-3.579288,"Stock(conId=14217, symbol='SIE', right='0', pr...",...,,False,,176.932007,-6.567993,EUR,0.00,DU3180649,180.907990,1.00000
36,VOW,,STK,2024-05-29 11:27,765.0,53.865656,140.970450,139.963074,-0.714601,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,107071.751419,-1.007376,EUR,-3230.09,DU3180649,109196.092541,1.00000
37,VOW,,STK,2024-05-29 11:27,-941.0,-64.969265,141.000179,139.963074,0.735535,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-131705.252399,975.916251,EUR,0.00,DU3180649,-131705.252399,1.00000


In [13]:
ac

Arctic(config=S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb))

In [15]:
df = pm.get_ib_positions_for_gui()
df

0     37.0
1   -699.0
2      1.0
3   -176.0
Name: position, dtype: float64
          timestamp symbol asset class  position   % of nav  averageCost  \
0  2024-05-29 12:21   AAPL         STK      37.0   3.191450   180.260541   
1  2024-05-29 12:21    CNR         STK    -699.0 -39.411599   174.287153   
2  2024-05-29 12:21    SIE         STK       1.0   0.087447   183.500000   

   marketPrice     pnl % strategy  \
0   189.779999  5.280944            
1   169.473999  2.761623            
2   177.360001 -3.346049            

                                            contract  ... close_dt deleted  \
0  Stock(conId=265598, symbol='AAPL', right='0', ...  ...            False   
1  Stock(conId=4458894, symbol='CNR', right='0', ...  ...            False   
2  Stock(conId=14217, symbol='SIE', right='0', pr...  ...            False   

  delete_dt  marketValue unrealizedPNL  currency  realizedPNL    account  \
0                7021.86        352.22       USD          0.0  DU3180649   
1     

StreamDescriptorMismatch: The columns (names and types) in the argument are not identical to that of the existing version: APPEND 
(Showing only the mismatch. Full col list saved in the `last_mismatch_msg` attribute of the lib instance.
'-' marks columns missing from the argument, '+' for unexpected.)
+FD<name=symbol, type=TD<type=UTF_DYNAMIC64, dim=0>>
+FD<name=strategy, type=TD<type=UTF_DYNAMIC64, dim=0>>
+FD<name=asset class, type=TD<type=UTF_DYNAMIC64, dim=0>>
-FD<name=symbol, type=TD<type=UTF_DYNAMIC64, dim=0>>
-FD<name=asset class, type=TD<type=UTF_DYNAMIC64, dim=0>>
-FD<name=strategy, type=TD<type=UTF_DYNAMIC64, dim=0>>

In [102]:
df_ac = ac.get_library('portfolio').read('positions').data
# Filter out deleted entries before comparing
df_ac_active = df_ac[df_ac['deleted'] != True].copy()
df_ac_active
#latest_positions_in_ac = df_ac_active.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()

NoSuchVersionException: E_NO_SUCH_VERSION read_dataframe_version: version matching query 'latest' not found for symbol 'positions'

In [109]:
symbol = "XLU"
asset_class = 'Put 63.0 20240419'
position = -1
strategy = ''


def delete_symbol(symbol, asset_class, position, strategy):
    # Load the library and read the existing DataFrame
    lib = ac.get_library('portfolio')
    df = lib.read('positions').data

    # Ensure 'deleted' and 'delete_dt' columns exist
    if 'deleted' not in df.columns:
        df['deleted'] = False
    if 'delete_dt' not in df.columns:
        df['delete_dt'] = None

    # Filter for matching entries that have not been previously marked as deleted
    filter_condition = (df['symbol'] == symbol) & \
                       (df['asset class'] == asset_class) & \
                       (df['position'] == float(position)) & \
                       (df['strategy'] == strategy) & \
                       (df['deleted'] == False)

    # Mark the filtered entries as deleted and add the current timestamp
    df.loc[filter_condition, 'deleted'] = True
    df.loc[filter_condition, 'delete_dt'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Convert to string

    # Save the updated DataFrame back to ArcticDB
    if 'positions' in lib.list_symbols():
        df = normalize_columns(df)
        lib.write('positions', df, prune_previous_versions=True)
        print(f"Deleted positions for {symbol} {asset_class} with position {position}")
    else:
        print("Positions symbol not found in library.")
            
def normalize_columns(df):
    df = df.copy()
    df['contract'] = df['contract'].astype(str)
    df['trade'] = df['trade'].astype(str)
    return df

# df = ac.get_library('portfolio').read('positions').data
# df

In [114]:
lib = ac.get_library('portfolio')
df = lib.read('positions').data
filter_condition = (df['symbol'] == symbol) & \
                       (df['asset class'] == asset_class) & \
                       (df['position'] == float(position)) & \
                       (df['strategy'] == strategy) & \
                       (df['deleted'] == False)

df[filter_condition]

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
63,XLU,,Put 63.0 20240419,2024-03-24 13:20,-1.0,-0.050033,86.1997,0.568817,34.011684,"Option(conId=652393383, symbol='XLU', lastTrad...",...,None,False,,-56.88,29.32,USD,0.0,U7706434,-52.625375,1.080847


In [115]:
delete_symbol(symbol,asset_class,position,strategy)
df = ac.get_library('portfolio').read('positions').data
df

[2024-03-24 13:24:13.744] [arcticdb] [info] Column close_dt does not have non null elements.


Deleted positions for XLU Put 63.0 20240419 with position -1


,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,BABA,test,STK,2024-03-23 23:43,30.0,1.904720,76.181877,72.180000,-5.253056,"Stock(conId=166090175, symbol='BABA', right='0...",...,None,True,2024-03-23 23:45:19,2165.400000,-120.060000,USD,0.0,U7706434,2003.428051,1.080847
1,EWT,,STK,2024-03-23 23:43,100.0,4.281969,44.256829,48.680000,9.994325,"Stock(conId=253190576, symbol='EWT', right='0'...",...,None,False,,4868.000000,442.320000,USD,0.0,U7706434,4503.873535,1.080847
2,EWT,,Call 50.0 20240419,2024-03-23 23:43,-1.0,-0.046540,48.960000,0.529119,-8.071691,"Option(conId=684551409, symbol='EWT', lastTrad...",...,None,False,,-52.910000,-3.950000,USD,0.0,U7706434,-48.952331,1.080847
3,IAU,,STK,2024-03-23 23:43,300.0,10.806104,38.454255,40.950100,6.490427,"Stock(conId=474829650, symbol='IAU', right='0'...",...,None,False,,12285.030000,748.750000,USD,0.0,U7706434,11366.109591,1.080847
4,IAU,,Call 39.0 20240419,2024-03-23 23:43,-1.0,-0.188853,44.950000,2.146970,-377.635106,"Option(conId=648748170, symbol='IAU', lastTrad...",...,None,False,,-214.700000,-169.750000,USD,0.0,U7706434,-198.640437,1.080847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,TLT,,Put 92.0 20240419,2024-03-24 13:20,-2.0,-0.000885,96.159600,0.502863,47.705393,"Option(conId=626935162, symbol='TLT', lastTrad...",...,None,False,,-1.005726,-95.656737,USD,0.0,U7706434,-0.930497,1.080847
60,TLT,,Put 94.0 20240419,2024-03-24 13:20,-2.0,-0.002187,113.309450,1.243414,-9.736152,"Option(conId=626935337, symbol='TLT', lastTrad...",...,None,False,,-2.486829,-112.066036,USD,0.0,U7706434,-2.300814,1.080847
61,TLT,,Call 95.0 20240419,2024-03-24 13:20,-1.0,-0.000780,68.949800,0.886399,-28.557160,"Option(conId=626933014, symbol='TLT', lastTrad...",...,None,False,,-0.886399,-68.063401,USD,0.0,U7706434,-0.820096,1.080847
62,TLT,,Call 97.0 20240426,2024-03-24 13:20,-2.0,-0.000807,46.798750,0.458580,2.010235,"Option(conId=689139606, symbol='TLT', lastTrad...",...,None,False,,-0.917160,-46.340170,USD,0.0,U7706434,-0.848556,1.080847


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfarm.nj; eufarm; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfarm.nj; eufarm; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederherg

In [60]:
latest_positions_in_ac = df_ac_active.groupby(['symbol', 'strategy', 'asset class']).apply(lambda x: x.sort_values(by='timestamp', ascending=False).head(1)).reset_index(drop=True)
latest_positions_in_ac

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,BABA,,STK,2024-03-23 18:23,30.0,1.904720,76.181877,72.180000,-5.253056,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-03-23,None,False,2165.400009,-4.001876,USD,0.0,U7706434,2003.428059,1.080847
1,EWT,,Call 50.0 20240419,2024-03-23 18:23,-1.0,-0.000465,48.960000,0.529119,-8.071691,"Option(conId=684551409, symbol='EWT', lastTrad...",...,2024-03-23,None,False,-0.529119,-48.430881,USD,0.0,U7706434,-0.489541,1.080847
2,EWT,,STK,2024-03-23 18:23,100.0,4.281969,44.256829,48.680000,9.994325,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-03-23,None,False,4868.000030,4.423171,USD,0.0,U7706434,4503.873563,1.080847
3,IAU,,Call 39.0 20240419,2024-03-23 18:23,-1.0,-0.001889,44.950000,2.146970,-377.635106,"Option(conId=648748170, symbol='IAU', lastTrad...",...,2024-03-23,None,False,-2.146970,-42.803030,USD,0.0,U7706434,-1.986376,1.080847
4,IAU,,STK,2024-03-23 18:23,300.0,10.806104,38.454255,40.950100,6.490427,"Stock(conId=474829650, symbol='IAU', right='0'...",...,2024-03-23,None,False,12285.029985,2.495845,USD,0.0,U7706434,11366.109578,1.080847
5,INDA,,STK,2024-03-23 18:23,100.0,4.440467,49.712499,50.481899,1.547700,"Stock(conId=101484826, symbol='INDA', right='0...",...,2024-03-23,None,False,5048.189925,0.769400,USD,0.0,U7706434,4670.585251,1.080847
6,IUSQ,,STK,2024-03-23 18:23,100.0,7.171358,65.602785,75.430000,14.979875,"Stock(conId=239363241, symbol='IUSQ', right='0...",...,2024-03-23,None,False,7543.000030,9.827215,EUR,0.0,U7706434,7543.000030,1.000000
7,MTUM,,Put 175.0 20240419,2024-03-23 18:23,1.0,0.000517,154.996900,0.587429,-62.100581,"Option(conId=684706174, symbol='MTUM', lastTra...",...,2024-03-23,None,False,0.587429,-154.409471,USD,0.0,U7706434,0.543490,1.080847
8,PRX,,STK,2024-03-23 18:23,250.0,6.593314,26.561554,27.740000,4.436658,"Stock(conId=382625193, symbol='PRX', right='0'...",...,2024-03-23,None,False,6934.999937,1.178445,EUR,0.0,U7706434,6934.999937,1.000000
9,QQQ,,STK,2024-03-23 18:23,10.0,3.920447,447.040500,445.700012,-0.299858,"Stock(conId=320227571, symbol='QQQ', right='0'...",...,2024-03-23,None,False,4457.000122,-1.340488,USD,0.0,U7706434,4123.616453,1.080847


In [49]:
lib = ac.get_library('portfolio')
positions_df = lib.read('positions').data
positions_df

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,BABA,,STK,2024-03-23 12:26,30.0,1.904720,76.181877,72.180000,-5.253056,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-03-23,None,False,2165.400000,-120.060000,USD,0.0,U7706434,2003.428051,1.080847
1,EWT,,STK,2024-03-23 12:26,100.0,4.281969,44.256829,48.680000,9.994325,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-03-23,None,False,4868.000000,442.320000,USD,0.0,U7706434,4503.873535,1.080847
2,EWT,,Call 50.0 20240419,2024-03-23 12:26,-1.0,-0.046540,48.960000,0.529119,-8.071691,"Option(conId=684551409, symbol='EWT', lastTrad...",...,2024-03-23,None,False,-52.910000,-3.950000,USD,0.0,U7706434,-48.952331,1.080847
3,IAU,,STK,2024-03-23 12:26,300.0,10.806104,38.454255,40.950100,6.490427,"Stock(conId=474829650, symbol='IAU', right='0'...",...,2024-03-23,None,False,12285.030000,748.750000,USD,0.0,U7706434,11366.109591,1.080847
4,IAU,,Call 39.0 20240419,2024-03-23 12:26,-1.0,-0.188853,44.950000,2.146970,-377.635106,"Option(conId=648748170, symbol='IAU', lastTrad...",...,2024-03-23,None,False,-214.700000,-169.750000,USD,0.0,U7706434,-198.640437,1.080847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,TLT,,Put 94.0 20240419,2024-03-23 15:10,-2.0,-0.002187,113.309450,1.243414,-9.736152,"Option(conId=626935337, symbol='TLT', lastTrad...",...,2024-03-23,None,False,-2.486829,-112.066036,USD,0.0,U7706434,-2.300814,1.080847
95,TLT,,Call 95.0 20240419,2024-03-23 15:10,-1.0,-0.000780,68.949800,0.886399,-28.557160,"Option(conId=626933014, symbol='TLT', lastTrad...",...,2024-03-23,None,False,-0.886399,-68.063401,USD,0.0,U7706434,-0.820096,1.080847
96,TLT,,Call 97.0 20240426,2024-03-23 15:10,-2.0,-0.000807,46.798750,0.458580,2.010235,"Option(conId=689139606, symbol='TLT', lastTrad...",...,2024-03-23,None,False,-0.917160,-46.340170,USD,0.0,U7706434,-0.848556,1.080847
97,XLU,test,Put 63.0 20240419,2024-03-23 15:10,-1.0,-0.000500,86.199700,0.568817,34.011684,"Option(conId=652393383, symbol='XLU', lastTrad...",...,2024-03-23,None,False,-0.568817,-85.630883,USD,0.0,U7706434,-0.526270,1.080847


In [30]:
lib.delete('positions')

[2024-03-23 12:21:00.301] [arcticdb] [warning] Symbol: positions does not exist.


### Create an Instance of PortfolioManager

In [17]:
pm = PortfolioManager(ib)

#### Get the current positions from your portfolio by strategy as assigned in ArcticDB

In [29]:
# df = pm.match_ib_positions_with_arcticdb()
# df.tail()
pm.delete_symbol_from_portfolio_lib('positions')

[2024-03-23 12:20:30.514] [arcticdb] [warning] Symbol: positions does not exist.


In [ ]:
ac.get_library('portfolio').delete('positions')

Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: cashfarm; usfarm.nj; eufarm; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: cashfarm; usfarm.nj; eufarm; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren 

#### Functions
- to assign a position to a strategy
- to delete a position in ArcticDB 
(will be written again to acdb if still in portfolio, but should lose its strategy assignment )

In [27]:
def assign_strategy(df, idx, strategy):
    # Get the symbol from the row
    if idx in df.index:
        symbol = df.loc[idx,'symbol']
        asset_class = df.loc[idx,'asset class']
        position = df.loc[idx,'position']

        filter_cond = (df['symbol'] == symbol) & \
                  (df['asset class'] == asset_class) & \
                  (df['position'] == position)
        
        filtered_df = df.loc[filter_cond]

        if not filtered_df.empty:
            # Update the strategy for the filtered rows
            df.loc[filter_cond, 'strategy'] = strategy
        
        pm.library.write('positions', df, metadata={'updated': 'strategy assignment from gui'})

def delete_strategy(df,idx,strategy):
    
        
assign_strategy(df,8,'SVIX')

In [28]:
df

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,BABA,,STK,2024-02-19 20:58,30.0,1.985980,76.181877,73.820000,-3.100313,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-02-19,None,False,2214.599991,-2.361877,USD,0.0,U7706434,2054.931791,NaN
1,EWT,,STK,2024-02-19 20:58,200.0,8.266396,44.256828,46.090000,4.142122,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-02-19,None,False,9218.000030,1.833172,USD,0.0,U7706434,8553.400789,NaN
2,IAU,,STK,2024-02-19 20:58,300.0,10.247381,38.454255,38.090099,-0.946983,"Stock(conId=474829650, symbol='IAU', right='0'...",...,2024-02-19,None,False,11427.029805,-0.364155,USD,0.0,U7706434,10603.163965,NaN
3,INDA,,STK,2024-02-19 20:58,100.0,4.592693,49.712499,51.213901,3.020171,"Stock(conId=101484826, symbol='INDA', right='0...",...,2024-02-19,None,False,5121.390150,1.501402,USD,0.0,U7706434,4752.148232,NaN
4,IUSQ,,STK,2024-02-19 20:58,100.0,7.019294,65.602785,72.629997,10.711759,"Stock(conId=239363241, symbol='IUSQ', right='0...",...,2024-02-19,None,False,7262.999725,7.027212,EUR,0.0,U7706434,7262.999725,1.0
5,PRX,,STK,2024-02-19 20:58,250.0,6.937123,26.561554,28.711903,8.095717,"Stock(conId=382625193, symbol='PRX', right='0'...",...,2024-02-19,None,False,7177.975650,2.150348,EUR,0.0,U7706434,7177.975650,1.0
6,TLT,,STK,2024-02-19 20:58,300.0,25.017103,97.181951,92.989998,-4.313510,"Stock(conId=15547841, symbol='TLT', right='0',...",...,2024-02-19,None,False,27896.999355,-4.191954,USD,0.0,U7706434,25885.681873,NaN
7,TLT,,Put 93.0 20240223,2024-02-19 20:58,-1.0,-0.000516,93.959600,0.575791,38.719290,"Option(conId=676006202, symbol='TLT', lastTrad...",...,2024-02-19,None,False,-0.575791,-93.383809,USD,0.0,U7706434,-0.534278,NaN
8,VXM,SVIX,VXMJ4 20240417,2024-02-19 20:58,-3.0,-0.042884,1615.525000,15.940211,1.331076,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-19,None,False,-47.820634,-1599.584789,USD,0.0,U7706434,-44.372862,NaN


#### Delete 'positions' from library portfolio

In [51]:
#pm.delete_symbol_from_portfolio_lib('positions')
ac.get_library('portfolio').delete('positions')

In [ ]:
#pm.library.read('positions').data
ac.get_library('portfolio').read('positions').data

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,BABA,,STK,2024-02-19 22:44,30.0,1.986228,76.181877,73.820000,-3.100313,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-02-19,None,False,2214.600000,-70.860000,USD,0.0,U7706434,2054.931799,1.0777
0,EWT,,STK,2024-02-19 22:44,200.0,8.267428,44.256828,46.090000,4.142122,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-02-19,None,False,9218.000000,366.630000,USD,0.0,U7706434,8553.400761,1.0777
0,IAU,,STK,2024-02-19 22:44,300.0,10.248660,38.454255,38.090099,-0.946983,"Stock(conId=474829650, symbol='IAU', right='0'...",...,2024-02-19,None,False,11427.030000,-109.250000,USD,0.0,U7706434,10603.164146,1.0777
0,INDA,,STK,2024-02-19 22:44,100.0,4.593266,49.712499,51.213901,3.020171,"Stock(conId=101484826, symbol='INDA', right='0...",...,2024-02-19,None,False,5121.390000,150.140000,USD,0.0,U7706434,4752.148093,1.0777
0,IUSQ,,STK,2024-02-19 22:44,100.0,7.020170,65.602785,72.629997,10.711759,"Stock(conId=239363241, symbol='IUSQ', right='0...",...,2024-02-19,None,False,7263.000000,702.720000,EUR,0.0,U7706434,7263.000000,1.0000
0,PRX,,STK,2024-02-19 22:44,250.0,6.937993,26.561554,28.711903,8.095717,"Stock(conId=382625193, symbol='PRX', right='0'...",...,2024-02-19,None,False,7177.980000,537.590000,EUR,0.0,U7706434,7177.980000,1.0000
0,TLT,,STK,2024-02-19 22:44,300.0,25.020225,97.181951,92.989998,-4.313510,"Stock(conId=15547841, symbol='TLT', right='0',...",...,2024-02-19,None,False,27897.000000,-1257.590000,USD,0.0,U7706434,25885.682472,1.0777
0,TLT,,Put 93.0 20240223,2024-02-19 22:44,-1.0,-0.051642,93.959600,0.575791,38.719290,"Option(conId=676006202, symbol='TLT', lastTrad...",...,2024-02-19,None,False,-57.580000,36.380000,USD,0.0,U7706434,-53.428598,1.0777
0,VXM,SVIX,VXMJ4 20240417,2024-02-19 22:44,-3.0,-4.288928,1615.525000,15.940211,1.331076,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-19,None,False,-4782.060000,64.510000,USD,0.0,U7706434,-4437.283103,1.0777
0,BABA,,STK,2024-02-19 22:44,30.0,1.986228,76.181877,73.820000,-3.100313,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-02-19,None,False,2214.599991,-2.361877,USD,0.0,U7706434,2054.931799,1.0777


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfuture.nj; usfarm.nj; eufarm; cashfarm; usopt; usfarm; euhmds; ushmds; secdefeu.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfuture.nj; usfarm.nj; eufarm; cashfarm; usopt; usfarm; euhmds; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME%